In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import os
from IPython.display import clear_output, Image, display, HTML

from problems.dataset import Dataset
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# load in graph etc
folder = "logs/mnist/u/08_15/20/logs"
file = "model_mnist_u.ckpt-10.meta"
meta = os.path.join(folder,file)

In [4]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.compat.v1.train.import_meta_graph(meta)

In [5]:
sess = tf.compat.v1.Session()

In [6]:
saver.restore(sess, meta[:-5])

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from logs/mnist/u/08_15/20/logs/model_mnist_u.ckpt-10


In [7]:
tensors = [n.name for n in tf.get_default_graph().as_graph_def().node]

In [8]:
[ten for ten in tensors if "Identity" in ten]

['rnn/while/Identity',
 'rnn/while/Identity_1',
 'rnn/while/Identity_2',
 'rnn/while/Identity_3',
 'Identity']

In [9]:
# construct default graph
graph = tf.get_default_graph()

In [10]:
output_o = graph.get_tensor_by_name("Identity:0")
output_o

<tf.Tensor 'Identity:0' shape=(?, 10) dtype=float32>

In [11]:
init_state = graph.get_tensor_by_name("init_state:0")
init_state

<tf.Tensor 'init_state:0' shape=(?, 1024) dtype=float32>

In [12]:
input_x = graph.get_tensor_by_name("input_x_mnist_u:0")
input_x

<tf.Tensor 'input_x_mnist_u:0' shape=(?, ?, 1) dtype=float32>

In [13]:
# input_x = tf.placeholder(tf.float32, [None, None, 1], name="input_x_mnist_u")

In [14]:
input_images = np.reshape(mnist.test.images, [-1, 784, 1])

In [47]:
# select only subset to work with (for now) of size 8
max_batch_size = 160
inds = np.random.randint(0, len(input_images), max_batch_size)
input_subset = input_images[inds,:,:]

In [48]:
num_hidden = 128
batch_size = len(input_subset)
init_state_C = np.sqrt(3 / (2 * num_hidden))

In [49]:
batch_size

160

In [50]:
init_state_val = np.random.uniform(-init_state_C, 
                                   init_state_C, 
                                   [batch_size, init_state.shape[1]])

In [51]:
feed_dict = {input_x: input_subset, init_state: init_state_val}

In [52]:
logits = output_o.eval(session=sess, feed_dict=feed_dict)

In [53]:
print(logits)

[[ 0.04043451  0.1349034  -0.22918248 ... -0.02504203  0.14375958
  -0.17502378]
 [-0.0037858  -0.21179411 -0.07525579 ...  0.02965734  0.14723909
  -0.12185239]
 [-0.0819393   0.02838133 -0.12648231 ...  0.010441    0.19825885
   0.03299588]
 ...
 [ 0.21697074 -0.00689475 -0.16650803 ... -0.09567403  0.02524315
  -0.1560786 ]
 [ 0.62372667 -0.017224   -0.17291218 ...  0.518902    0.902282
  -0.17606115]
 [ 0.21756199  0.05786519 -0.0872687  ...  0.15630819  0.08686144
  -0.34566072]]


In [54]:
# check that logits map to sensible predictions
pred = tf.nn.softmax(logits)
print(pred.eval(session=sess))
pred_classes = np.argmax(np.array(pred.eval(session=sess)), axis=1)
print(pred_classes)

[[0.10024829 0.11018039 0.07655681 ... 0.09389466 0.1111605  0.08081736]
 [0.09772639 0.0793734  0.09098563 ... 0.10104993 0.11365829 0.08684327]
 [0.0887688  0.09912243 0.08490153 ... 0.09736    0.11747591 0.0995809 ]
 ...
 [0.12439308 0.09944265 0.08477221 ... 0.09099475 0.10269044 0.08566096]
 [0.13772252 0.07255103 0.06209107 ... 0.1240167  0.18196172 0.06189585]
 [0.12329648 0.10509819 0.0909001  ... 0.11597075 0.10819025 0.07020146]]
[6 6 8 3 0 7 3 3 8 0 8 1 8 0 4 6 8 8 3 0 6 7 8 7 1 1 0 0 7 0 1 8 9 1 8 3 0
 8 8 8 0 8 3 8 7 6 6 0 0 8 6 6 0 0 0 8 4 0 8 3 7 6 6 4 6 8 0 7 1 7 9 7 6 6
 6 3 3 8 7 6 8 6 8 4 0 0 0 3 3 8 3 3 3 1 8 6 5 9 0 7 7 8 7 3 3 8 3 8 0 0 8
 6 8 0 6 6 7 8 7 6 6 0 4 0 7 8 8 0 0 9 3 9 0 3 1 0 6 3 8 6 0 0 4 8 8 6 2 0
 0 6 4 0 7 9 6 4 7 0 8 0]


In [55]:
# check that all digits are represented in this sample
list(set(pred_classes))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
from adversarial.logits import CheckpointInference

In [5]:
cpt = CheckpointInference(folder, 10)

AssertionError: Checkpoint does not exist.

In [6]:
folder

'logs/mnist/u/08_15/20/logs'

In [72]:
cpt.get_logits(input_subset)

array([[ 0.04030069,  0.13489881, -0.22919889, ..., -0.02521045,
         0.14367038, -0.17513338],
       [-0.00404576, -0.21177065, -0.07534543, ...,  0.02951844,
         0.14703026, -0.12203041],
       [-0.08163646,  0.02836534, -0.12639454, ...,  0.01044099,
         0.19823131,  0.03287729],
       ...,
       [ 0.21685934, -0.00664707, -0.16599949, ..., -0.09583103,
         0.02514534, -0.15609299],
       [ 0.6231816 , -0.01627101, -0.17371638, ...,  0.5178762 ,
         0.90387636, -0.17682208],
       [ 0.21691734,  0.05796039, -0.08677098, ...,  0.1567448 ,
         0.08749337, -0.3454393 ]], dtype=float32)

In [59]:
print(pred_classes)
print(cpt.get_predictions(input_subset))

[6 6 8 3 0 7 3 3 8 0 8 1 8 0 4 6 8 8 3 0 6 7 8 7 1 1 0 0 7 0 1 8 9 1 8 3 0
 8 8 8 0 8 3 8 7 6 6 0 0 8 6 6 0 0 0 8 4 0 8 3 7 6 6 4 6 8 0 7 1 7 9 7 6 6
 6 3 3 8 7 6 8 6 8 4 0 0 0 3 3 8 3 3 3 1 8 6 5 9 0 7 7 8 7 3 3 8 3 8 0 0 8
 6 8 0 6 6 7 8 7 6 6 0 4 0 7 8 8 0 0 9 3 9 0 3 1 0 6 3 8 6 0 0 4 8 8 6 2 0
 0 6 4 0 7 9 6 4 7 0 8 0]
[6 6 8 3 0 7 3 3 8 0 8 1 8 0 4 6 8 8 3 0 6 7 8 7 1 1 0 0 7 0 1 8 9 1 8 3 0
 8 8 8 0 8 3 8 7 6 6 0 0 8 6 6 0 0 0 8 4 0 8 3 7 6 6 4 6 8 0 7 1 7 9 7 6 6
 6 3 3 8 7 6 8 6 8 4 0 0 0 3 3 8 3 3 3 1 8 6 5 9 0 7 7 8 7 3 3 8 3 8 0 0 8
 6 8 0 6 6 7 8 7 6 6 0 4 0 7 8 8 0 0 9 3 9 0 3 1 0 6 3 8 6 0 0 4 8 8 6 2 0
 0 6 4 0 7 9 6 4 7 0 8 0]


In [60]:
from adversarial.fast_gradient_method import fast_gradient_method as fgm

In [73]:
single_input = input_images[420,:,:]

In [74]:
single_logits = cpt.get_logits(single_input)

ValueError: Cannot feed value of shape (784, 1) for Tensor 'input_x_mnist_u:0', which has shape '(?, ?, 1)'

In [68]:
single_logits[0].shape

(10,)

In [63]:
eps = 0.001
norm = 2
adversarial_output = fgm(cpt.get_logits, single_input, eps, norm)

SystemError: <built-in function TFE_Py_TapeWatch> returned a result with an error set

In [12]:
from cleverhans.attacks import FastGradientMethod

In [14]:
FastGradientMethod

cleverhans.attacks.FastGradientMethod

In [18]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)
  print(t.watched_variables())

()


In [19]:
dz_dx = t.gradient(z, x)

In [20]:
dz_dx

<tf.Tensor 'Tile:0' shape=(2, 2) dtype=float32>